In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import acquire
import prepare_311


import re
from datetime import timedelta as td, datetime as dt

%matplotlib inline

In [2]:
df_311 = acquire.acquire_311()

## Prepare

## Get 311 for prepare

In [3]:
df_311 = prepare_311.prepare_311(df_311)

In [4]:
df_311.category.value_counts()

Streets & Infrastructure    1677
Name: category, dtype: int64

In [5]:
df_311.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1677 entries, 8362 to 304395
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype          
---  ------                          --------------  -----          
 0   category                        1677 non-null   object         
 1   case_id                         1677 non-null   int64          
 2   late                            1677 non-null   object         
 3   dept                            1677 non-null   object         
 4   reason_name                     1677 non-null   object         
 5   event_name                      1677 non-null   object         
 6   case_status                     1677 non-null   object         
 7   sourceid                        1677 non-null   object         
 8   event_address                   1677 non-null   object         
 9   council_district                1677 non-null   int64          
 10  report_start_date               1677 non-null   object 

In [6]:
df_311.shape

(1677, 22)

In [7]:
df_311.sample(30)

,category,case_id,late,dept,reason_name,event_name,case_status,sourceid,event_address,council_district,...,zip_code,reported_date,closed_date,due_date,case_days_length,report_to_assigned_days_length,days_past_due_length,30_days,60_days,90_days
243135,Streets & Infrastructure,1015906068,NO,Trans & Cap Improvements,Storm Water,Drainage Inlet - Cleaning/Foul Odor,Closed,Constituent Call,"13526 PUEBLO CROSSIN, San Antonio, 78232",9,...,78232,2019-10-31,2019-11-15 00:00:00.000000,2019-11-22,15 days 00:00:00,22,15,True,False,False
89050,Streets & Infrastructure,1015703949,NO,Trans & Cap Improvements,Storm Water,Drainage Channel/Debris Removal,Closed,Constituent Call,"1807 ADAMSTON DR, San Antonio, 78220",2,...,78220,2019-07-22,2019-07-25 00:00:00.000000,2019-09-03,3 days 00:00:00,43,3,True,False,False
151326,Streets & Infrastructure,1015788116,NO,Trans & Cap Improvements,Storm Water,Drainage Channel/Debris Removal,Closed,Constituent Call,"4018 FLOWING PATH, San Antonio, 78247",10,...,78247,2019-08-30,2019-09-05 00:00:00.000000,2019-10-14,6 days 00:00:00,45,6,True,False,False
18938,Streets & Infrastructure,1015542925,NO,Trans & Cap Improvements,Storm Water,Drainage Channel/Debris Removal,Closed,Web Portal,"5772 SUN CANYON DR, San Antonio, 78244",2,...,78244,2019-05-21,2019-07-03 00:00:00.000000,2019-07-03,43 days 00:00:00,43,43,False,True,False
118500,Streets & Infrastructure,1015743765,NO,Trans & Cap Improvements,Storm Water,Drainage Investigation,Closed,Web Portal,"19510 ENCINO KNOLL, San Antonio, 78259",9,...,78259,2019-08-09,2019-10-14 00:00:00.000000,2019-10-14,66 days 00:00:00,66,66,False,False,False
232903,Streets & Infrastructure,1015893736,NO,Trans & Cap Improvements,Storm Water,Drainage Channel/Debris Removal,Closed,Constituent Call,"2000 EL MONTE BLVD, San Antonio, 78201",1,...,78201,2019-10-24,2019-10-25 00:00:00.000000,2019-12-10,1 days 00:00:00,47,1,True,False,False
163263,Streets & Infrastructure,1015804091,NO,Trans & Cap Improvements,Storm Water,Drainage Channel/Debris Removal,Closed,Constituent Call,"5214 HILLBURN DR, San Antonio, 78242",4,...,78242,2019-09-09,2019-10-18 00:00:00.000000,2019-10-21,39 days 00:00:00,42,39,False,True,False
29589,Streets & Infrastructure,1015619665,NO,Trans & Cap Improvements,Storm Water,Drainage Investigation,Closed,Constituent Call,"215 BICKLEY, San Antonio, 78221",3,...,78221,2019-06-18,2019-06-26 00:00:00.000000,2019-08-21,8 days 00:00:00,64,8,True,False,False
193399,Streets & Infrastructure,1015843799,NO,Trans & Cap Improvements,Storm Water,Drainage Channel/Debris Removal,Closed,Constituent Call,"3567 FREDSBG RD, San Antonio, 78201",7,...,78201,2019-09-27,2019-10-03 00:00:00.000000,2019-11-08,6 days 00:00:00,42,6,True,False,False
106993,Streets & Infrastructure,1015728142,NO,Trans & Cap Improvements,Storm Water,Drainage Channel/Debris Removal,Closed,Web Portal,"7735 NW 410, San Antonio, 78245",6,...,78245,2019-08-01,2019-08-02 00:00:00.000000,2019-09-13,1 days 00:00:00,43,1,True,False,False


In [ ]:
df_311.dept.value_counts()

In [ ]:
df_311.case_days_length.mean()

In [ ]:
df_311.report_to_assigned_days_length.mean()

# calls avg
- 30 days
- 60 days
- 90 days or more

In [ ]:
# df_311['days_past_due_length'] = (df_311.case_days_length.dt.days)

In [ ]:
# df_311['report_to_assigned_days_length'] = (df_311.report_to_assigned_days_length.dt.days)

In [ ]:
# df_311['30_days'] = df_311.case_days_length.dt.days <= 30
# df_311['60_days'] = ((df_311.case_days_length.dt.days > 30) & (df_311.case_days_length.dt.days <= 60))
# df_311['90_days'] = df_311.case_days_length.dt.days >= 90

In [ ]:
df_311.sample(25)

In [ ]:
df_311.head()

In [8]:
df_311.dept.value_counts()

Trans & Cap Improvements    1677
Name: dept, dtype: int64

In [9]:
df_311.reason_name.value_counts()

Storm Water    1570
Streets         107
Name: reason_name, dtype: int64

In [10]:
df_311.case_status.value_counts()

Closed    1655
Open        22
Name: case_status, dtype: int64

In [ ]:
case_status
